## API´s

In [30]:
import requests
import pandas as pd
import time
import os

In [31]:
#verificando API 1 WORLBANK (DATOS ECONOMICOS)
 
import requests

url = "https://api.worldbank.org/v2/country?format=json"
response = requests.get(url)

response.status_code


200

In [32]:
#Solictando datos en formato JSON
def fetch_api_data(url, params=None):
    response = requests.get(url, params=params)

    if response.status_code != 200:
        raise Exception(f"Error API: {response.status_code}")

    return response.json()


In [33]:
#json
data = response.json()

print(type(data))
print(len(data))


<class 'list'>
2


In [34]:
#Descargar PIB per cápita en World BAnk API para todos los paises 
url = "https://api.worldbank.org/v2/country/all/indicator/NY.GDP.PCAP.CD"

params = {
    "format": "json",
    "per_page": 20000
}

data = fetch_api_data(url, params)


In [35]:
#Convirtiendo datos para pandas,lista de registros.
records = data[1]

df_gdp = pd.DataFrame([{
    "country": r["country"]["value"] if r["country"] else None,
    "country_code": r["countryiso3code"],
    "year": int(r["date"]),
    "gdp_per_capita": r["value"]
} for r in records])

df_gdp.head()


,country,country_code,year,gdp_per_capita
0,Africa Eastern and Southern,AFE,2024,1615.396356
1,Africa Eastern and Southern,AFE,2023,1571.449189
2,Africa Eastern and Southern,AFE,2022,1679.327622
3,Africa Eastern and Southern,AFE,2021,1562.416175
4,Africa Eastern and Southern,AFE,2020,1351.591669


In [36]:
print(data[1][0])



{'indicator': {'id': 'NY.GDP.PCAP.CD', 'value': 'GDP per capita (current US$)'}, 'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'}, 'countryiso3code': 'AFE', 'date': '2024', 'value': 1615.39635562849, 'unit': '', 'obs_status': '', 'decimal': 1}


In [37]:
#Limpieza 
df_gdp = df_gdp.dropna(subset=["gdp_per_capita"])
df_gdp = df_gdp[df_gdp["year"] >= 2000]

df_gdp.info()


<class 'pandas.DataFrame'>
Index: 6433 entries, 0 to 17249
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country         6433 non-null   str    
 1   country_code    6433 non-null   str    
 2   year            6433 non-null   int64  
 3   gdp_per_capita  6433 non-null   float64
dtypes: float64(1), int64(1), str(2)
memory usage: 251.3 KB


In [38]:
#creando carpeta de datos procesados con la finalidad de mantener separadas las fuentes originales y los datasets modificados
import os

output_path = "../data/processed/worldbank_gdp.csv"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

df_gdp.to_csv(output_path, index=False)



In [39]:
#VERIFICANDO API 2 OPENALEX (PRODUCCION CIENTFICA IA)
import requests
import pandas as pd
import os



In [40]:

#Llamada a open alex
url = "https://api.openalex.org/works"
params = {
    "search": "artificial intelligence ethics",
    "per-page": 100
}

response = requests.get(url, params=params)
response.status_code



200

In [41]:
#Conversi{on a Data Frame 
data = response.json()
records = data.get("results", [])

df_openalex = pd.DataFrame([
    {
        "title": r.get("title"),
        "year": r.get("publication_year"),
        "citations": r.get("cited_by_count"),
        "open_access": r.get("open_access", {}).get("is_oa"),
        "journal": r.get("host_venue", {}).get("display_name"),
        "country": (
            r.get("authorships", [{}])[0]
            .get("institutions", [{}])[0]
            .get("country_code")
            if r.get("authorships") and r["authorships"][0].get("institutions")
            else None
        )
    }
    for r in records
])


In [42]:
df_openalex.head()
df_openalex.shape


(100, 6)

In [43]:
#Guardando local 
os.makedirs("../data/processed", exist_ok=True)
df_openalex.to_csv("../data/processed/openalex_ai.csv", index=False)


In [44]:
#Filtro por conceptos y palabras clave

url = "https://api.openalex.org/works"

params = {
    "search": "artificial intelligence ethics",
    "per-page": 200,
    "mailto": "student@example.com"  # no obligatorio, pero recomendado
}

data = requests.get(url, params=params).json()


In [45]:
#Convirtiendo resultados para Dataframe
#Esto es nuevo aprendido, la finalidad es crear una funcion segura para extaer el país isn romper el Data

def get_country_from_openalex(record):
    try:
        authorships = record.get("authorships", [])
        if not authorships:
            return None

        institutions = authorships[0].get("institutions", [])
        if not institutions:
            return None

        return institutions[0].get("country_code")

    except Exception:
        return None



In [46]:
# Con ayuda, reconstruyendo Data, esto para dejar una version "limpia"
df_openalex = pd.DataFrame([{
    "title": r.get("title"),
    "year": r.get("publication_year"),
    "citations": r.get("cited_by_count"),
    "open_access": r.get("open_access", {}).get("is_oa"),
    "journal": r.get("host_venue", {}).get("display_name"),
    "country": get_country_from_openalex(r)
} for r in records])


In [47]:
#LIMPIEZA
df_openalex = df_openalex.dropna(subset=["year"])
df_openalex = df_openalex[df_openalex["year"] >= 2000]

df_openalex.info()


<class 'pandas.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        100 non-null    str   
 1   year         100 non-null    int64 
 2   citations    100 non-null    int64 
 3   open_access  100 non-null    bool  
 4   journal      0 non-null      object
 5   country      88 non-null     str   
dtypes: bool(1), int64(2), object(1), str(2)
memory usage: 4.1+ KB


In [48]:
#Incluso en bases de datos abiertas sobre investigación científica en IA, existe una pérdida significativa de información, lo que dificulta análisis precisos y refuerza la necesidad de estándares de datos.
#El analisis se realizará únicamente sobre las publicaciones de afiliacion identificable.

In [49]:
df_openalex["country"].isna().mean()

0.27

print ("27% de artículos que no tiene país asignable")

27% de artículos que no tiene país asignable


In [50]:
#Aproximadamente el 27 % de las publicaciones obtenidas desde OpenAlex no incluyen información de país a nivel institucional. 
#Este fenómeno refleja limitaciones estructurales en los metadatos académicos y se tiene en cuenta al interpretar los análisis comparativos por país.
#Realizando la investigación, se ha visto que los papers sin país sueles ser colaboraciones, prepints o instituciones privadas.
#Así mismo, es importante yaq ue no tomarlo en cuenta puede sobre-representar a paises ricos ó visceversa
